## Notebook A

#### Get initial data to the right form for the pipeline

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/ART_output/ICE_MOstrains.csv')
df = df[[' Name*',' Summary*']]
# Rename columns
df.columns = ['Line Name', 'Line Description']
df.tail()

,Line Name,Line Description
91,Strain 92,ACCOAC_0_MDH_0_PTAr_2_CS_2_ACACT1r_0_PPC_2_PPC...
92,Strain 93,ACCOAC_1_MDH_2_PTAr_0_CS_0_ACACT1r_0_PPC_2_PPC...
93,Strain 94,ACCOAC_0_MDH_2_PTAr_0_CS_1_ACACT1r_0_PPC_2_PPC...
94,Strain 95,ACCOAC_0_MDH_1_PTAr_2_CS_0_ACACT1r_1_PPC_1_PPC...
95,WT,Wild type E. coli


In [3]:
iso = pd.read_csv('../data/omg_output/edd/EDD_isoprenol_production.csv')
iso.tail()

,Line Name,Measurement Type,Value,Time,Units
91,Strain 92,CID:12988,0.00000,9.0,mM
92,Strain 93,CID:12988,0.00000,9.0,mM
93,Strain 94,CID:12988,0.00000,9.0,mM
94,Strain 95,CID:12988,0.00000,9.0,mM
95,WT,CID:12988,0.46188,9.0,mM


In [5]:
df = pd.merge(df, iso, on='Line Name', how='outer')
df = df[['Line Name', 'Line Description', 'Measurement Type', 'Value']]
df.head()

,Line Name,Line Description,Measurement Type,Value
0,Strain 1,ACCOAC_1_MDH_1_PTAr_2_CS_0_ACACT1r_2_PPC_0_PPC...,CID:12988,0.000000
1,Strain 2,ACCOAC_1_MDH_2_PTAr_2_CS_2_ACACT1r_2_PPC_1_PPC...,CID:12988,0.552101
2,Strain 3,ACCOAC_1_MDH_0_PTAr_0_CS_2_ACACT1r_1_PPC_1_PPC...,CID:12988,0.349196
3,Strain 4,ACCOAC_1_MDH_1_PTAr_1_CS_1_ACACT1r_2_PPC_2_PPC...,CID:12988,0.551849
4,Strain 5,ACCOAC_2_MDH_0_PTAr_2_CS_1_ACACT1r_1_PPC_0_PPC...,CID:12988,0.080117


### Adding design information to the df

In [6]:
reactions = df['Line Description'][0].split('_')[::2]
for rxn in reactions:
    df[rxn] = None

for i in range(len(df)):
    if df['Line Name'][i]=='WT':
        for r in range(len(reactions)):
            df.iloc[i, (4+r)] = float(1)
    else:
        values = df.loc[i]['Line Description'].split('_')[1::2]
        for r,value in zip(range(len(reactions)),values):
            df.iloc[i, (4+r)] = float(value)

df = df.drop(columns='Line Description')

df.tail()

,Line Name,Measurement Type,Value,ACCOAC,MDH,PTAr,CS,ACACT1r,PPC,PPCK,PFL
91,Strain 92,CID:12988,0.00000,0.0,0.0,2.0,2.0,0.0,2.0,0.0,2.0
92,Strain 93,CID:12988,0.00000,1.0,2.0,0.0,0.0,0.0,2.0,2.0,0.0
93,Strain 94,CID:12988,0.00000,0.0,2.0,0.0,1.0,0.0,2.0,1.0,1.0
94,Strain 95,CID:12988,0.00000,0.0,1.0,2.0,0.0,1.0,1.0,0.0,2.0
95,WT,CID:12988,0.46188,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [9]:
# Save to csv
df.to_csv('../data/EDD_isoprenol_production.csv', index=False)